<a href="https://colab.research.google.com/github/nkinesis/soen471-project/blob/main/Comparison_DTRFBT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


In [39]:
# datasets = ['regular', 'undersample']
# metrics = ['f1_score']
# models = ['dt', 'rs', 'bs']

# for ds in datasets:
#     # train test split blabla
#     for metric in metrics:
#         for model in models:
#             # load model from pickle whatever
#             if model is 'dt'
#               pickled_model = pickle.load(open('DT_model.pickle', 'rb'))
#             # compute f1 score for that loaded model
#             f1_score = 0.88
#             print(f1_score)
#             # generate chart
#             plt.save('mychart.png')

In [40]:
#loading the dataset
df = pd.read_csv('london_clean.csv')
# df.dtypes

In [41]:
# load the data, do train test split (copy from the notebook)
X = df[['DateOfCall', 'PropertyType', 'NumPumpsAttending', 'PumpHoursRoundUp', 'mean_temp']]
y = df[['CostCat']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# print a small sample of train X and y
print(X_train[0:5])
print(y_train[0:5])


        DateOfCall  PropertyType  NumPumpsAttending  PumpHoursRoundUp  \
233987          10             8                  2                 2   
639721           8            78                  1                 1   
588384           2           155                  1                 1   
489135           2            75                  3                 4   
820835           6            22                  1                 1   

        mean_temp  
233987        8.0  
639721       17.3  
588384        5.6  
489135        1.2  
820835       18.7  
        CostCat
233987      2.0
639721      0.0
588384      0.0
489135      4.0
820835      1.0


In [42]:
# load the models
pickled_model_dt = pickle.load(open('DT_model.pickle', 'rb'))
pickled_model_rf = pickle.load(open('RF_model.pickle','rb'))
pickled_model_bt = pickle.load(open('boosted_trees_best_full.pickle','rb'))

[04:23:31] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [43]:
# generate predictions
y_pred_dt = pickled_model_dt.predict(X_test)
y_pred_rf = pickled_model_rf.predict(X_test)
y_pred_bt = pickled_model_bt.predict(X_test)


In [44]:
# compute the f1 score (if you want to compute other metrics, just change this step, the rest is the same)
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score

# f1_score = pickled_model.f1_score(y_test, y_pred)
f1_score_dt = f1_score(y_test, y_pred_dt, average='weighted')
f1_score_dt

f1_score_rf = f1_score(y_test, y_pred_rf, average='weighted')
f1_score_rf

f1_score_bt = f1_score(y_test, y_pred_bt, average='weighted')
f1_score_bt

0.8850922008142844

In [47]:
# write the code to draw your graph
import plotly.express as px

classifiers = ['Decision Tree Classifier','Random Forest Classifier','Boosted Tree Classifier']
metrics = ['F1-Score']

df = pd.DataFrame({'Classifier': [classifiers[0]] + [classifiers[1]] + [classifiers[2]] ,
                   'Metrics': metrics*3,
                   'Score': [f1_score_dt,f1_score_rf,f1_score_bt]})

fig = px.bar(df, x='Classifier', y='Score', color='Metrics', barmode='group',
             height=400, width=600, title="Classifier Performance Metrics")

fig.update_layout(xaxis=dict(categoryorder='category ascending'))
fig.update_traces(customdata=df['Score'],texttemplate='%{customdata}', textposition='auto', textfont=dict(size=8))
fig.update_traces(text=df['Score'], texttemplate='%{text}', textposition='outside', textfont=dict(size=8))
fig.update_traces(hovertemplate='%{x}<br>Score: %{y:.4f}<extra></extra>')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_traces(text=df['Score'].round(2).astype(str), texttemplate='%{text:.4f}', textposition='outside', textfont=dict(size=8))
fig.show()